<a href="https://colab.research.google.com/github/yskmt2018/quantum/blob/master/knapsack_problem_sqa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Knapsack Problem with Simulated Quantum Annealing

* This notebook was created and tested on Google Colaboratory. (2020/09)

## Reference

> https://openjij.github.io/OpenJijTutorial/build/html/ja/8-KnapsackPyqubo.html

In [1]:
!python -V
!pip install --quiet pyqubo openjij

Python 3.6.9


In [2]:
import random
import pyqubo
from openjij import SQASampler

## Problem Settings

* 各荷物の価格のリスト$c$と重量のリスト$w$がある。

* 選んだ荷物の合計重量が、ナップサックの最大容量$W$以下となる制約を満たしながら、選んだ荷物の合計価格を最大化する。


In [3]:
def random_baggage(baggage_number):
  c = {}
  w = {}
  for i in range(baggage_number):
    c[i] = random.randrange(W/2, W)
    w[i] = random.randrange(1, W/2)
  
  return c, w

In [4]:
# Knapsack Capacity
W = 20
# Baggage Number
N = 10
# c: Cost List, w: Weight List
c, w = random_baggage(N)

print('c : {}'.format(c))
print('w : {}'.format(w))

c : {0: 13, 1: 18, 2: 16, 3: 13, 4: 10, 5: 13, 6: 18, 7: 16, 8: 13, 9: 14}
w : {0: 8, 1: 7, 2: 1, 3: 3, 4: 6, 5: 9, 6: 8, 7: 9, 8: 6, 9: 7}


## Formulation

* KP の目的関数及び制約条件を定式化し、QUBO（Quadratic Unconstrained Binary Optimization）式を構築する。

* QUBO 式の構築には、OSS ライブラリ PyQUBO（[GitHub](https://github.com/recruit-communications/pyqubo), [Documentation](https://pyqubo.readthedocs.io/en/latest/)）を用いる。

### QUBO1: Objective Function

* 目的関数：荷物の合計価格

$$
-\sum^{N-1}_{\alpha = 0} c_{\alpha}x_{\alpha}
$$

In [5]:
def build_objective(x):
  H = - sum(c[a] * x[a] for a in range(N))
  return H

### QUBO2: Capacity rule

* 制約条件：荷物の合計重量が、ナップサックの最大容量以下となる。

$$
w\left(W - \sum^{N-1}_{\alpha = 0}w_{\alpha}x_{\alpha} - Y \right)^2
$$

In [6]:
def build_capacity_rule(x, y):
  H = pyqubo.Constraint((W - sum(w[a] * x[a] for a in range(N)) - y)**2, 'w')
  return H

### Hamiltonian

* 目的関数及び制約条件からハミルトニアン H を以下のように定義する。

$$
H = -\sum^{N-1}_{\alpha = 0} c_{\alpha}x_{\alpha}
+ w\left(W - \sum^{N-1}_{\alpha = 0}w_{\alpha}x_{\alpha} - Y \right)^2
$$

In [7]:
# Selected Baggage
x = pyqubo.Array.create('x', shape=(N), vartype='BINARY')
# Slack Variable
y = pyqubo.LogEncInteger('y', 0, W)

In [8]:
H = build_objective(x) + \
    pyqubo.Placeholder('w') * build_capacity_rule(x, y)

In [9]:
feed_dict = {'w': 1}

In [10]:
model = H.compile()
qubo, constant = model.to_qubo(feed_dict=feed_dict)

## Execute Quantum Annealing

* Sampler と呼ばれるモジュールを生成し、構築した QUBO 式を渡すことで、量子アニーリングを実行する。

* 結果は、Response オブジェクトとして返却される。

### OpenJij

* OpenJij（[GitHub](https://github.com/OpenJij/OpenJij), [Documentation](https://openjij.github.io/OpenJij_Documentation/build/html/), [Tutorial](https://openjij.github.io/OpenJijTutorial/build/html/ja/index.html)）

* OSS として、量子アニーリングをシミュレートする SQA（Simulated Quantum Annealing）の Python 用 API を提供している。

In [11]:
sampler = SQASampler(num_reads=10)

In [12]:
%%time
response = sampler.sample_qubo(qubo)

CPU times: user 122 ms, sys: 714 µs, total: 123 ms
Wall time: 128 ms


### Take Solutions

* Response オブジェクトに格納されている量子アニーリングの結果（解）を取り出す。

In [13]:
def extract_samples(response):
  solutions = []
  energies = []
  
  for record in response.record:
    sol, num_occ = record[0], record[2]
    solution, broken, energy = model.decode_solution(dict(zip(response.variables, sol)), vartype='BINARY', feed_dict=feed_dict)
    if len(broken) == 0:
      solutions += [solution] * num_occ
      energies += [energy] * num_occ
  
  return solutions, energies

In [14]:
solutions, energies = extract_samples(response)
best_solution = solutions[energies.index(min(energies))]

### Print Knapsack

* ナップサックに入れた荷物の合計価格と合計重量を表示する。

* 先に設定した制約条件が満たされていることを確認してください。
> 荷物の合計重量が、ナップサックの最大容量以下となる。

In [15]:
baggage_name = [chr(b) for b in range(65, 65+N)]
baggage_name

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']

In [16]:
total_cost = 0
total_weight = 0

print('Selected Baggages:')
for (k, v) in best_solution['x'].items():
    if v == 1:
        print(' {} cost={}, weight={}'.format(baggage_name[k], c[k], w[k]))
        total_cost += c[k]
        total_weight += w[k]

print()
print('Total Cost={}, Total Weight={}, Knapsack Capacity={}'.format(total_cost, total_weight, W))

Selected Baggages:
 B cost=18, weight=7
 C cost=16, weight=1
 D cost=13, weight=3
 G cost=18, weight=8

Total Cost=65, Total Weight=19, Knapsack Capacity=20
